In [9]:
import numpy as np
import pandas as pd
import xarray as xr
from joblib import dump, load
from sklearn.model_selection import train_test_split
import math as mt
from math import e
from sklearn.preprocessing import StandardScaler

In [10]:
# Read Orography
OroData = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5IGP_Orography.nc')

In [11]:
#Surface data
t2mData = xr.open_dataset('../../../Data/CMIP6/EC-Earth3/processed/Regrid_tas_day_EC-Earth3_historical_r1i1p1f1_gr_18500101-20141231.nc')
rhData  = xr.open_dataset('../../../Data/CMIP6/EC-Earth3/processed/Regrid_hurs_day_EC-Earth3_historical_r1i1p1f1_gr_18500101-20141231.nc')
u10Data = xr.open_dataset('../../../Data/CMIP6/EC-Earth3/processed/Regrid_uas_day_EC-Earth3_historical_r1i1p1f1_gr_18500101-20141231.nc')
v10Data = xr.open_dataset('../../../Data/CMIP6/EC-Earth3/processed/Regrid_vas_day_EC-Earth3_historical_r1i1p1f1_gr_18500101-20141231.nc')

In [12]:
# Level Data
tLevData = xr.open_dataset('../../../Data/CMIP6/EC-Earth3/processed/Regrid_ta_day_EC-Earth3_historical_r1i1p1f1_gr_18500101-20141231.nc')
zLevData = xr.open_dataset('../../../Data/CMIP6/EC-Earth3/processed/Regrid_zg_day_EC-Earth3_historical_r1i1p1f1_gr_18500101-20141231.nc')
wLevData = xr.open_dataset('../../../Data/CMIP6/EC-Earth3/processed/Regrid_wap_day_EC-Earth3_historical_r1i1p1f1_gr_19750101-20141231.nc')
uLevData = xr.open_dataset('../../../Data/CMIP6/EC-Earth3/processed/Regrid_ua_day_EC-Earth3_historical_r1i1p1f1_gr_18500101-20141231.nc')

In [13]:
t2mD = t2mData.sel(latitude=slice(32,24),longitude=slice(74,86))
u10D = u10Data.sel(latitude=slice(32,24),longitude=slice(74,86))
v10D = v10Data.sel(latitude=slice(32,24),longitude=slice(74,86))
rhD  = rhData.sel(latitude=slice(32,24),longitude=slice(74,86))
tLevD = tLevData.sel(latitude=slice(32,24),longitude=slice(74,86))
uLevD = uLevData.sel(latitude=slice(32,24),longitude=slice(74,86))
w700D = wLevData.sel(latitude=slice(32,24),longitude=slice(74,86)).sel(plev=70000,method='nearest')
zLevD = zLevData.sel(latitude=slice(32,24),longitude=slice(74,86))
OroD = OroData.sel(latitude=slice(32,24),longitude=slice(74,86))
#w700D

In [14]:
# Calculate wind speed and relative humidity inv  ushear
ws = ((v10D.vas.values**2)+(u10D.uas.values**2))**0.5
ws_ds = xr.Dataset({'ws': (('time','latitude','longitude'), ws)},
                   coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})

rh_ds = xr.Dataset({'rh': (('time','latitude','longitude'), rhD.hurs)},
                   coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})

#Calculate inv
inv=t2mD.tas.values-tLevD.ta.sel(plev=85000,method='nearest').values
inv_ds = xr.Dataset({'inv': (('time','latitude','longitude'), inv)}, 
                    coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})
inv_ds.attrs
inv_ds.attrs['units']='K'
inv_ds.attrs['long_name']='t2m - t850'

#u shear calculation
ushear=(uLevD.ua.sel(plev=85000,method='nearest').values-u10D.uas.values)/(zLevD.zg.sel(plev=85000,method='nearest').values) 
ushear_ds = xr.Dataset({'ushear': (('time','latitude','longitude'), ushear)}, 
                       coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})
ushear_ds.attrs['units']='s-1'
ushear_ds.attrs['long_name']='(u10 - u850)/z850'

In [16]:
AOData = xr.open_dataset('EC-Earth3-AOindex-NDJF-Daily-1980-2014.nc')

AO=AOData.AO
#datetimeindex = AO.indexes['time'].to_datetimeindex()#
#datetimeindex
#AO['time'] = datetimeindex
AO.values

AO5D=AO.rolling(time=5).mean()

AO5DAll=AO5D[((AO5D.time.dt.month>11) | (AO5D.time.dt.month<2)) & 
             (AO5D.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
#ao

In [17]:
EUData = xr.open_dataset('EC-Earth3-EUindex-NDJF-Daily-1980-2014.nc')
EU=EUData.EU
#datetimeindex = EU.indexes['time'].to_datetimeindex()
#datetimeindex
#EU['time'] = datetimeindex
EU.values
EU5D=EU.rolling(time=5).mean()

EU5DAll=EU5D[((EU5D.time.dt.month>11) | (EU5D.time.dt.month<2)) & 
             (EU5D.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))


In [18]:
# create mask
oro = OroD.z
oro.values = OroD.z.values/9.81
oro.attrs
oro.attrs['units']='meter'
oro.attrs['long_name']='Orography'
oro.values[oro.values>500.1]=np.NaN
mask=oro.values/oro.values

In [19]:
# Apply Mask 
t2mD.tas.values=t2mD.tas.values*mask
ws_ds.ws.values=ws_ds.ws.values*mask
rh_ds.rh.values=rh_ds.rh.values*mask
w700D.wap.values=w700D.wap.values*mask
inv_ds.inv.values=inv_ds.inv.values*mask
ushear_ds.ushear.values=ushear_ds.ushear.values*mask

In [20]:
# Feild mean
t2mTS1=t2mD.mean(dim=['latitude','longitude'], skipna=True).tas
wsTS1=ws_ds.mean(dim=['latitude','longitude'], skipna=True).ws
rhTS1=rh_ds.mean(dim=['latitude','longitude'], skipna=True).rh
wTS1=w700D.mean(dim=['latitude','longitude'], skipna=True).wap
invTS1=inv_ds.mean(dim=['latitude','longitude'], skipna=True).inv
ushearTS1=ushear_ds.mean(dim=['latitude','longitude'], skipna=True).ushear

In [21]:
# shift one day so we can take values of previous day
t2mTS=t2mTS1.shift(time=1)
wsTS=wsTS1.shift(time=1)
rhTS=rhTS1.shift(time=1)
wTS=wTS1.shift(time=1)
invTS=invTS1.shift(time=1)
ushearTS=ushearTS1.shift(time=1)

In [22]:
# Select required time 
t2m=t2mTS[((t2mTS.time.dt.month>11) | (t2mTS.time.dt.month<2)) & (t2mTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
ws=wsTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
inv=invTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
rh=rhTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
w=wTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
ushear=ushearTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))

In [23]:
X       = pd.DataFrame([t2m.values,ws.values,rh.values/100.0,inv.values,w.values,ushear.values*100.0,AO5DAll.values,
                        EU5DAll.values],
                       index=['t2m', 'ws', 'rh','inv', 'w', 'ushear', 'AO5D', 'EU5D']).T[:]
X.index = pd.to_datetime(t2m.time.values)
X.describe(include='all')

,t2m,ws,rh,inv,w,ushear,AO5D,EU5D
count,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000
mean,287.924082,2.206321,0.468928,3.732428,0.008479,0.116737,-0.230842,-0.005231
std,2.439329,0.591917,0.166504,1.890626,0.094583,0.105905,0.905442,0.567736
min,280.323578,0.853991,0.122970,-3.622917,-0.599347,-0.289089,-3.192107,-1.879477
25%,286.224548,1.786189,0.338896,2.668484,-0.036142,0.050352,-0.879229,-0.412850
50%,288.072464,2.124089,0.451544,4.061439,0.024140,0.122064,-0.271585,0.019818
75%,289.664375,2.562951,0.591668,5.150329,0.067259,0.188245,0.392925,0.403062
max,294.511810,5.775387,0.875731,7.834291,0.237297,0.487323,2.329217,1.398473


In [24]:
norm = StandardScaler().fit(X)
    # transform training data
X = norm.transform(X)
X
X = pd.DataFrame(norm.transform(X))
X.index = pd.to_datetime(t2m.time.values)
X.describe(include='all')

,0,1,2,3,4,5,6,7
count,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000
mean,-118.061338,-3.728274,-2.816973,-1.974631,-0.089671,-1.102534,0.255008,0.009216
std,0.410138,1.690204,6.008640,0.529169,10.577649,9.446766,1.104942,1.762193
min,-119.339253,-7.589817,-15.301604,-4.033326,-68.065936,-37.302365,-3.358725,-5.808238
25%,-118.347090,-4.927951,-7.509460,-2.272419,-5.079953,-7.024110,-0.536240,-1.255988
50%,-118.036390,-3.963087,-3.444328,-1.882544,1.661702,-0.627350,0.205289,0.086966
75%,-117.768733,-2.709928,1.612358,-1.577773,6.483903,5.275966,1.016213,1.276515
max,-116.953708,6.463098,11.863350,-0.826556,25.500171,31.953818,3.379137,4.366164


In [25]:
regLin=load('../../March2021/Observation_models/LRModel.joblib') 

In [26]:
yLR=regLin.predict(X)
y_predLin_ds=xr.Dataset({'yLR': (('time'), yLR)}, coords={'time': t2m.time})

In [27]:
dump(y_predLin_ds.yLR,'../Model_plots/EC-Earth3-LR-Y.joblib')

['../Model_plots/EC-Earth3-LR-Y.joblib']

In [28]:
regLin2=load('../../March2021/Observation_models/SVRModel.joblib') 

In [29]:
regLin2=load('../../March2021/Observation_models/SVRModel.joblib') 
yLR=regLin2.predict(X)
y_predLin_ds=xr.Dataset({'yLR': (('time'), yLR)}, coords={'time': t2m.time})
dump(y_predLin_ds.yLR,'../Model_plots/EC-Earth3-SVR-Y.joblib')

['../Model_plots/EC-Earth3-SVR-Y.joblib']